# Model fit on one experimental trial #

This notebook was originally designed to reproduce figure 4g of the paper: the BLS model comparison across all behavioral sessions, where the authors fitted both Bayesian timing models to each session and compared their goodness of fit using the Mean Squared Error (MSE) between empirical and model-predicted timing behavior. 

Because we only have access to one empirical trial per monkey (from the data accompanying Figure 1b), we cannot recreate the session-level comparison in Fig. 4g. Instead, we apply the same fitting procedures and compute analogous metrics on the single available trial for each animal.

In [1]:
import pandas as pd 
import numpy as np 
from Identifiability_test import CounterModel, NonCounterModel

**Data description:** 

The Figure 1b data files provide two vectors:
- $v_a(t)$ : actual velocity
- $v_p(t)$ : predicted velocity 

Velocity is signed, but its magnitude corresponds to the timing interval. We extract vectors $t_s, t_p$ where $t_s$ is the true interval encoded in the trial and $t_p$ is the produced interval  

In [2]:
file = pd.ExcelFile(r"C:\Users\frdes\Documents\Neuroscience\Project\mental-navigation\src\mental_navigation\Fig1\Data_Fig1.xlsx")
df = pd.read_excel(file, sheet_name="fig_1b")
df.head() 

,nhp_id,va_mnav,vp_mnav,Unnamed: 3,nhp_id.1,va_mnav.1,vp_mnav.1
0,a,-0.65,-0.416667,NaN,m,2.60,1.483333
1,a,-0.65,-0.550000,NaN,m,0.65,0.783333
2,a,-1.95,-1.366666,NaN,m,2.60,2.233334
3,a,-1.95,-1.383333,NaN,m,1.95,2.416644
4,a,-1.95,-0.550000,NaN,m,0.65,0.833333


In [3]:
df_a= df[['va_mnav','vp_mnav']].copy()
df_m = df[['va_mnav.1','vp_mnav.1']].copy()
df_m = df_m.rename(columns= {'va_mnav.1':'va_mnav','vp_mnav.1':'vp_mnav'})
print(df_a.head())
print(df_m.head())
df_a = df_a.dropna(subset=['va_mnav','vp_mnav'])
print('Samples monkey A:',len(df_a))
print(df_a.isna().sum())
df_m = df_m.dropna(subset=['va_mnav','vp_mnav'])
print('Samples monkey M:',len(df_m))
print(df_m.isna().sum())

   va_mnav   vp_mnav
0    -0.65 -0.416667
1    -0.65 -0.550000
2    -1.95 -1.366666
3    -1.95 -1.383333
4    -1.95 -0.550000
   va_mnav   vp_mnav
0     2.60  1.483333
1     0.65  0.783333
2     2.60  2.233334
3     1.95  2.416644
4     0.65  0.833333
Samples monkey A: 1028
va_mnav    0
vp_mnav    0
dtype: int64
Samples monkey M: 1382
va_mnav    0
vp_mnav    0
dtype: int64


In [4]:
# I will use absolute velocities as times 
ts_a = df_a["va_mnav"].to_numpy()
tp_a = df_a["vp_mnav"].to_numpy()

ts_m = df_m["va_mnav"].to_numpy()
tp_m = df_m["vp_mnav"].to_numpy()

# abs to match format required by my code (as in the Matlab one)
ts_a = np.abs(ts_a)
tp_a = np.abs(tp_a)
ts_m = np.abs(ts_m)
tp_m = np.abs(tp_m)

For both the **Counter model** (with reset) and the **Non-Counter model** (without reset) we estimate parameters  $$ \theta = (w_m,w_p,offset) $$ on the actual data using maximum likelihood. We then computed standard goodnes-of-fit measures like:
- Negative log-likelihood $ L $
- BIC: $$ \text{BIC} = kln(n) + 2(-log L)$$ 
- Single-trial MSE: the simple squared error between true values and one noisy model-generated sample (using the best fit parameters), to see how close the sample is to actual trial. It is a stochastic error measure, multiple runs will result in different MSE values
$$ MSE_{trial}= (t_p^{empirical}- t_p^{model})^2$$

The single trial MSE differs from the MSE in the paper that we computed and is explained later on.  

The first model we fit is the Counter model using data from monkey A.

In [ ]:
counter= CounterModel()

res_counter= counter.fit(ts_a,tp_a,w_init = (0.5,0.5,0.0))

# Recover the fitted parameters computed in .fit
w_opt = res_counter.x
wm_opt, wp_opt, offset_opt = w_opt 
print('Fitted parameters (wm,wp,offset):', w_opt)

# Negative log-likelihood at optimum
negloglik_opt= counter.neg_log_likelihood(w_opt,ts_a,tp_a)
print('Negative log-likelihood at optimum:',negloglik_opt)

# BIC (as in the Matlab code)
n= len(ts_a)
k= len(w_opt)
bic = np.log(n)*k + 2*negloglik_opt
print('BIC:',bic)

The following cell computes the $MSE_{trial}$. 

In the sample run for **Counter model** on data from monkey **A** this value amounts to $MSE_{trial} = 0.49116851529704697 $

In [ ]:
rng = np.random.default_rng(0) 

# Generate model predictions
tm_gen, te_gen, tp_gen = counter.simulate(ts_a, wm_opt, wp_opt, offset_opt, rng=rng)
mse_trial = np.mean((tp_a - tp_gen)**2)

print("MSE (trial-wise) =", mse_trial)

MSE (trial-wise) = 0.49116851529704697


In the paper, for each session, authors compute:
- the **mean empirical produced time** $\bar{t}_p(t_s)$ which is simply the mean of all produced intervals $ t_p $ that are relative to a specific time 'bin' $ts$ , $ts \in (0.65, 1.30, 1.95, 2.6, 3.25)$ 
$$
\bar{t}_p(t_s) = \frac{1}{n_s} \sum_{i : t_{s,i} = t_s} t_{p,i}
$$

- the **mean model-predicted produced time** $\hat{t}_p(t_{s})$, whose computation in explained below
- The **MSE** over the 5 base intervals 
$$
\mathrm{MSE}_{\text{session}}
= \frac{1}{5} \sum_{j=1}^{5}
\left( \bar{t}_p(t_{s,j}) - \hat{t}_p(t_{s,j}) \right)^2
$$

This is a deterministic model-data comparison of the mean curves, not of individual samples 

The mean model predicted produced interval  is obtained from the expected value of $t_p$ conditioned on a given base interval $t_s$:
$$
\hat{t}_p(t_s) = \mathbb{E}\big[\, t_p \mid t_s, \theta \,\big]
$$

Since the BLS generative model includes measurement noise and production noise, this is not available in closed form. 
Therefore, we approximate it using Monte Carlo sampling: 
1) For each unique value of $t_s$
2) Simulate a large batch of trial ($n= 5000$) collecting the resulting $t_p$ values 
3) Take their mean, so the approximation would be: 
$$
\hat{t}_p(t_s) \approx \frac{1}{N} \sum_{i=1}^{N} t_{p,i}^{\text{model}}
$$

We then use this approximation in the **$MSE_{session}$** computation 

In [41]:
def predict_tp_mean_per_ts_from_model(model, ts_empirical, wm, wp, offset,
                                      n_samples=5000, rng=None):
    """
    Approximate the model-predicted mean produced time for each unique ts
    by Monte Carlo using model.simulate.

    Parameters
    ----------
    model : BaseBayesianTimingModel (CounterModel or NonCounterModel)
    ts_empirical : 1D array of empirical ts from the session
    wm, wp, offset : fitted parameters for this model and this session
    n_samples : number of simulated trials per unique ts
    rng : numpy Generator (optional, for reproducibility)

    Returns
    -------
    unique_ts : 1D array of sorted unique ts values
    tp_model_mean : 1D array, same length as unique_ts
    tp_model_mean[i] ≈ E[tp | ts = unique_ts[i], theta]
    """
    ts_empirical = np.asarray(ts_empirical, dtype=float)
    print(ts_empirical)
    unique_ts = np.unique(ts_empirical)

    if rng is None:
        rng = np.random.default_rng(123)

    tp_model_mean = np.empty_like(unique_ts, dtype=float)

    for i, t in enumerate(unique_ts):
        # simulate n_samples trials all with the same ts = t
        ts_vec = np.full(n_samples, t, dtype=float)
        # Here it simulates the trials
        _, _, tp_sim = model.simulate(ts_vec, wm, wp, offset, rng=rng)
        print("ts =", t,
              "| any NaN in tp_sim?", np.isnan(tp_sim).any(),
              "| #NaN =", np.isnan(tp_sim).sum())
        if np.isnan(tp_sim).any():
            print('Warning: some samples returned Nan and were discarded from the mean computation')

        # Here it maps the interval to the mean 
        #tp_model_mean[i] = tp_sim.mean()
        tp_model_mean[i] = np.nanmean(tp_sim)
        
    print(tp_model_mean)
    return unique_ts, tp_model_mean

The following cell contains an helper function to compute the MSE given the true behavioural $t_p$  and the model mean $t_p$

In [42]:
def mse_means_only(ts, tp_data, tp_model_mean):
    """
    MSE between empirical mean tp(ts) and model-predicted mean tp(ts).
    """
    ts = np.asarray(ts)
    tp_data = np.asarray(tp_data)
    tp_model_mean = np.asarray(tp_model_mean)  # one value per unique ts

    unique_ts = np.unique(ts)
    mse_vals = []

    for i, t in enumerate(unique_ts):
        mask = (ts == t)
        data_mean = tp_data[mask].mean()
        model_mean = tp_model_mean[i]  # model gives one predicted mean per ts
        mse_vals.append((data_mean - model_mean)**2)

    return np.mean(mse_vals)


The following cells computes the **MSE** for the **Counter model**. This value comes out to be $$ MSE_{session} = 0.1252836794984097$$

In [44]:
# Previously fitted parameters are here: wm_opt, wp_opt, offset_opt = w_opt 

# generate model predicted mean 
unique_ts,tp_model_mean = predict_tp_mean_per_ts_from_model(
                                        model= counter,
                                        ts_empirical = ts_a, 
                                        wm = wm_opt,
                                        wp = wp_opt,
                                        offset = offset_opt,
                                        n_samples = 5000,
                                        rng= np.random.default_rng(0) 
                                    )

mse_val= mse_means_only(ts_a,tp_a,tp_model_mean)
print('MSE value of Counter model:', mse_val)

[0.65 0.65 1.95 ... 0.65 0.65 2.6 ]
ts = 0.65 | any NaN in tp_sim? True | #NaN = 32


c:\Users\frdes\Documents\Neuroscience\Project\mental-navigation\src\mental_navigation\Bayesianmodels\Identifyability.py:227: RuntimeWarning: invalid value encountered in sqrt
  return wp * np.sqrt(te)


ts = 1.3 | any NaN in tp_sim? True | #NaN = 3
ts = 1.95 | any NaN in tp_sim? True | #NaN = 1
ts = 2.6 | any NaN in tp_sim? False | #NaN = 0
ts = 3.25 | any NaN in tp_sim? False | #NaN = 0
[0.45995857 1.10442554 1.75601506 2.40046377 3.04575833]
MSE value of Counter model: 0.1252836794984097


In the following cells we reproduce the same computations using data from monkey **A** on **Non Counter model** . The trial $MSE$ for the Non Counter model is: 
$$ MSE_{trial} = 0.5152966061705184 $$

In [45]:
noncounter= NonCounterModel()

res_counter_n= noncounter.fit(ts_a,tp_a,w_init = (0.5,0.5,0.0))
# Recover the fitted parameters computed in .fit
w_opt_n = res_counter_n.x
wm_opt_n, wp_opt_n, offset_opt_n = w_opt_n 
print('Fitted parameters (wm,wp,offset):', w_opt_n)

# Negative log-likelihood at optimum
negloglik_opt_n= noncounter.neg_log_likelihood(w_opt_n,ts_a,tp_a)
print('Negative log-likelihood at optimum:',negloglik_opt_n)

# BIC (as in the Matlab code)
n_n= len(ts_a)
k_n= len(w_opt_n)
bic_n = np.log(n_n)*k_n + 2*negloglik_opt_n
print('BIC:',bic_n)

Fitted parameters (wm,wp,offset): [ 0.220947    0.23183494 -0.17881231]
Negative log-likelihood at optimum: 669.4253037232046
BIC: 1359.6567187844546


In [ ]:
rng = np.random.default_rng(0)
# Generate model predictions
tm_gen_n, te_gen_n, tp_gen_n = noncounter.simulate(ts_a, wm_opt_n, wp_opt_n, offset_opt_n, rng=rng)
mse_trial_n = np.mean((tp_a - tp_gen_n)**2)
print("MSE (trial-wise) =", mse_trial_n)

MSE (trial-wise) = 0.5152966061705184


In [ ]:
# Previously fitted parameters are here: wm_opt_n, wp_opt_n, offset_opt_n = w_opt_n 
# generate model predicted mean 
unique_ts_n,tp_model_mean_n = predict_tp_mean_per_ts_from_model(
                                        model= noncounter,
                                        ts_empirical = ts_a, 
                                        wm = wm_opt_n,
                                        wp = wp_opt_n,
                                        offset = offset_opt_n,
                                        n_samples = 5000,
                                        rng= np.random.default_rng(0) 
                                    )

mse_val_n= mse_means_only(ts_a,tp_a,tp_model_mean_n)
print('MSE value of NonCounter model:', mse_val_n)

[0.65 0.65 1.95 ... 0.65 0.65 2.6 ]
ts = 0.65 | any NaN in tp_sim? False | #NaN = 0
ts = 1.3 | any NaN in tp_sim? False | #NaN = 0
ts = 1.95 | any NaN in tp_sim? False | #NaN = 0
ts = 2.6 | any NaN in tp_sim? False | #NaN = 0
ts = 3.25 | any NaN in tp_sim? False | #NaN = 0
[0.47358655 1.12216718 1.77622295 2.41903607 3.06434245]
MSE value of NonCounter model: 0.12789651924728931


Here's the summary of the $MSE_{session}$ values obtained for the two models:
- Counter model: 
$$
MSE_{session}:  0.1252836794984097
$$
- Non Counter model: 
$$
MSE_{session}:  0.12789651924728931
$$

As these values are only for one session, we cannot reproduce any of the further statistical analyses and visualisations conducted in the paper.